In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 20.2 MB/s eta 0:00:00


In [2]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Model: YOLO11 Large
model = YOLO('yolo11l.pt')

100%|██████████| 49.0M/49.0M [00:00<00:00, 348MB/s]


In [4]:
# Sample Video Path
video_path = '/kaggle/input/videos/vietnam.mp4'

In [5]:
# Capture Video
cap = cv2.VideoCapture(video_path)

In [6]:
# Get Video's Properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

properties = {
    "Width": width, 
    "Height": height, 
    "FPS": fps}

for prop in properties:
    print(f"This is {video_path.split('/')[-1]} {prop}: {properties[prop]}")

This is vietnam.mp4 Width: 1920
This is vietnam.mp4 Height: 1080
This is vietnam.mp4 FPS: 59


In [7]:
video_name = video_path.split("/")[-1]  # Extract filename
output_path = f"run/{video_name.split('.')[0]}_tracked.mp4"  # Set correct file path

os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create "run/" directory if it doesn't exist

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Set codec
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Initialize VideoWriter

In [8]:
# Store History
track_history = defaultdict(lambda:[])

# Loop through Frames
while cap.isOpened():
    # Read a frame | Returs: [boolean], [frame]
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist = True, show = False)

        # Get Bounding Boxes and Track IDs
        bbox = results[0].boxes.xywh.cpu() # [x, y, w, h]
        try:
            track_ids = results[0].boxes.id
            if track_ids is not None:
                # Store all IDs of frame into a list
                track_ids = track_ids.int().cpu().tolist()
            else:
                track_ids = [] # No tracks found for this frame
        except AttributeError:
            tracK_ids = [] # Case Handling
        # Visualize the results
        annotated_frame = results[0].plot()

        # Plot the tracks if track exists
        if track_ids:
            # Loop through each object/track_id
            for bb, track_id in zip(bbox, track_ids):
                x, y, w, h = bb
                
                # Access the track_id History
                track = track_history[track_id] # Retrieve previous positions
                
                # Log x_center and y_center of track_id in each frame
                track.append((float(x), float(y))) # Store current position

                if len(track) > 120: # If a track_id exceeds 120 Instances/History
                    # Remove 1st Index
                    track.pop(0)

                # Draw tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2)) # Stack History of Motion Horizontally
                cv2.polylines(
                    annotated_frame,
                    [points],
                    isClosed = False,
                    color = (230, 230, 230),
                    thickness = 4
                )
        # Write frame to output video
        output.write(annotated_frame)
    else:
        break # Out of loop if fail to capture frame

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.4s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 19 persons, 6 cars, 26 motorcycles, 2 buss, 2 trucks, 91.3ms
Speed: 9.7ms preprocess, 91.3ms inference, 262.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 6 cars, 26 motorcycles, 1 bus, 2 trucks, 31.2ms
Speed: 2.5ms preprocess, 31.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 5 cars, 25 motorcycles, 1 bus, 3 trucks, 31.2ms
Speed: 3.0ms preprocess, 31.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 5 cars, 25 motorcycles, 1 bus, 4 trucks, 31.2ms
Speed: 2.6ms preprocess, 31.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 64

In [9]:
"""Example Iteration
Let's assume we have 3 frames in the video:

Frame	Objects Detected	Track IDs	Bounding Boxes (x, y, w, h)
1	Car, Person	[1, 2]	[(100, 200, 50, 60), (300, 400, 40, 80)]
2	Car, Person	[1, 2]	[(110, 210, 50, 60), (310, 410, 40, 80)]
3	Car, Person	[1, 2]	[(120, 220, 50, 60), (320, 420, 40, 80)]
Tracking History for Object ID 1 (Car)
Frame 1: track_history[1] = [(100, 200)]
Frame 2: track_history[1] = [(100, 200), (110, 210)]
Frame 3: track_history[1] = [(100, 200), (110, 210), (120, 220)]"""

"Example Iteration\nLet's assume we have 3 frames in the video:\n\nFrame\tObjects Detected\tTrack IDs\tBounding Boxes (x, y, w, h)\n1\tCar, Person\t[1, 2]\t[(100, 200, 50, 60), (300, 400, 40, 80)]\n2\tCar, Person\t[1, 2]\t[(110, 210, 50, 60), (310, 410, 40, 80)]\n3\tCar, Person\t[1, 2]\t[(120, 220, 50, 60), (320, 420, 40, 80)]\nTracking History for Object ID 1 (Car)\nFrame 1: track_history[1] = [(100, 200)]\nFrame 2: track_history[1] = [(100, 200), (110, 210)]\nFrame 3: track_history[1] = [(100, 200), (110, 210), (120, 220)]"

In [10]:
cap.release()
output.release()
print(f"Video has been saved to {output_path}")

Video has been saved to run/vietnam_tracked.mp4
